In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt 

%matplotlib inline

import warnings

warnings.filterwarnings("ignore")

from collections import Counter

/home/chen/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/chen/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/chen/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/chen/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/home/chen/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 

In [3]:
plt.rc("font", family="SimHei", size="15")

In [51]:
df = pd.read_csv("../data/criteo/criteo_data.csv")

In [5]:
from pyod.models.knn import KNN

In [8]:
continus = [f for f in df.columns if f.startswith("I")]

In [9]:
for col in continus:
    df[col].fillna(-999, inplace=True)

In [10]:
clf = KNN()

clf.fit(df[["I1", "I2"]].values)

KNN(algorithm='auto', contamination=0.1, leaf_size=30, method='largest',
  metric='minkowski', metric_params=None, n_jobs=1, n_neighbors=5, p=2,
  radius=1.0)

In [11]:
y_pred = clf.predict_proba(df[["I1", "I2"]].values)

In [12]:
y_pred

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [15]:
df["I1"].quantile([0.25, 0.5, 0.75])

0.25    0.0
0.50    1.0
0.75    3.0
Name: I1, dtype: float64

In [16]:
from scipy.stats import zscore

In [43]:
a = df["I1"].values

In [37]:
indices = np.array(list(map(lambda x: not x, np.isnan(df["I1"].values))))

In [38]:
indices

array([ True,  True,  True, ..., False,  True, False])

In [39]:
true_value = df["I1"].values[indices]

In [40]:
true_value

array([ 1.,  2.,  2., ...,  1.,  2., 26.])

In [41]:
z_values = zscore(true_value)

In [42]:
z_values

array([-0.2502643 , -0.1384593 , -0.1384593 , ..., -0.2502643 ,
       -0.1384593 ,  2.54486074])

In [44]:
a[indices] = z_values

In [46]:
a

array([-0.2502643 , -0.1384593 , -0.1384593 , ...,         nan,
        2.54486074,         nan])

In [58]:
def compute_zscore(df, features, threshold=2.5):
    
    for col in features:
        all_value = df[col].values.copy()
        ## 先剔除掉所有nan值
        indices = np.array(list(map(lambda x: not x, np.isnan(all_value))))
        true_value = all_value[indices]
        z_value = zscore(true_value)
        all_value[indices] = z_value
        ## 得到 zscore所在的列，以及是否是异常值的列
        df[col+"_zscore"] = all_value
        df[col+"_isanomaly"] = df[col+"_zscore"].apply(lambda x: int(abs(x) >= abs(threshold)))
        
    #return df

In [59]:
compute_zscore(df, ["I1"])

In [60]:
df.head()

,Label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C19,C20,C21,C22,C23,C24,C25,C26,I1_zscore,I1_isanomaly
0,0,1.0,1,5.0,0.0,1382.0,4.0,15.0,2.0,181.0,...,21ddcdc9,b1252a9d,07b5194c,NaN,3a171ecb,c5c50484,e8b83407,9727dd16,-0.250264,0
1,0,2.0,0,44.0,1.0,102.0,8.0,2.0,2.0,4.0,...,21ddcdc9,5840adea,60f6221e,NaN,3a171ecb,43f13e8b,e8b83407,731c3655,-0.138459,0
2,0,2.0,0,1.0,14.0,767.0,89.0,4.0,2.0,245.0,...,NaN,NaN,e587c466,ad3062eb,3a171ecb,3b183c5c,NaN,NaN,-0.138459,0
3,0,NaN,893,NaN,NaN,4392.0,NaN,0.0,0.0,0.0,...,NaN,NaN,6b3a5ca6,NaN,3a171ecb,9117a34a,NaN,NaN,NaN,0
4,0,3.0,-1,NaN,0.0,2.0,0.0,3.0,0.0,0.0,...,NaN,NaN,21c9516a,NaN,32c7478e,b34f3128,NaN,NaN,-0.026654,0


In [61]:
from pyod.models.iforest import IsolationForest

In [62]:
con_fe = [f for f in df.columns if f.startswith("I")]

In [63]:
X = df[con_fe].values

In [66]:
from sklearn.ensemble import IsolationForest

In [64]:
iso = IsolationForest()